# Vertical sections of estimated DIC and total alkalinity
Created by Ivan Lima on Tue Jan  4 2022 16:34:43 -0500

In [ ]:
import hvplot
import pandas as pd
import numpy as np
import holoviews as hv
import hvplot.pandas, datetime
from my_bokeh_themes import *
print('Last updated on {}'.format(datetime.datetime.now().ctime()))

## Read data

In [ ]:
df_wod = pd.read_hdf('data/bgc_all_2013-2019.h5', key='df_wod_bgc')
df_wod.head()

## Seasonal distribution of data

In [ ]:
tiles='EsriImagery'
p_surface = hv.Layout()
for season in ['winter','spring','summer','fall']:
    df = df_wod[df_wod.season==season]
    p = df.hvplot.points('Longitude', 'Latitude', geo=True, xlabel='longitude', ylabel='latitude', clabel='Alkalinity',
                         cmap='kbc', title='{}'.format(season.upper()), frame_width=400, rasterize=True, c='TA',
                         colorbar=True, coastline='50m', features={'land':'50m'})
    p_surface += p

p_surface.cols(2)

## Distribution of different types of data

In [ ]:
data_types = {1:'CTD', 2:'Glider', 3:'Float'}
plist = hv.Layout()
for t in sorted(data_types):
    df = df_wod[df_wod.Platform_Type==t]
    p = df.hvplot.points('Longitude', 'Latitude', geo=True, xlabel='longitude', ylabel='latitude', title='{} data'.format(data_types[t]),
                         frame_height=450, rasterize=True, c='TA', clabel='Alkalinity', cmap='kbc', colorbar=True,
                         coastline='50m', features={'land':'50m'})
    plist += p

plist.cols(2)

## Glider lines south of Cape Cod (Spring)

In [ ]:
df_glider = df_wod[df_wod.Platform_Type==2]
df = df_glider[(df_glider.Cast > 17616000) & (df_glider.Cast < 17622000)]
title = r'{} to {}'.format(df.Date.min().strftime('%F'), df.Date.max().strftime('%F'))
df.hvplot.points('Longitude', 'Latitude', geo=True, xlabel='longitude', ylabel='latitude', title=title, tiles=tiles,
                 frame_height=400, rasterize=False, c='TA', clabel='Alkalinity', cmap='kbc', colorbar=True,
                 hover_cols=['Cast','Date'])

In [ ]:
varinfo = {
    'Temperature': {
        'units':'degree C',
        'cmap':'bmy'},
    'Salinity': {
        'units':'g/kg',
        'cmap':'bgy'},
    'DIC': {
        'units':'micro mol/kg',
        'cmap':'bmw'},
    'TA': {
        'units':'micro mol/kg',
        'cmap':'bmw'},
    'pH': {
        'units':'',
        'cmap':'bmw'},
    'saturation_aragonite': {
        'units':'',
        'cmap':'bmw'}
}

def get_distance(df_in):
    """Compute distance along track in Km from latitude and logitude"""
    df = df_in.groupby('Cast').nth(0)
    df['Distance'] = np.sqrt((df.Latitude.diff()*111.12)**2 + (df.Longitude.diff()*111.12)**2).fillna(0).cumsum()
    df = df.reset_index()
    return pd.merge(df_in, df[['Cast','Distance']], on=['Cast'], how='inner')

def plot_sections(df, invert_x=True):
    title = r'{} / {}'.format(
        df.Date.min().strftime('%F'), df.Date.max().strftime('%F'))
    m = df.hvplot.points('Longitude', 'Latitude', geo=True, xlabel='longitude', ylabel='latitude', title=title,
                         tiles=tiles, frame_height=350, rasterize=False, c='red', hover_cols=['Cast','Date'], padding=(0.5, 0.25))
    plist = hv.Layout()
    for varname in ['Temperature', 'Salinity', 'DIC', 'TA', 'pH', 'saturation_aragonite']:
        p = df.hvplot.scatter(x='Distance', y='Depth', color=varname, frame_width=400, aspect=1.1, xaxis='top',
                              cmap=varinfo[varname]['cmap'], clabel=varinfo[varname]['units'], padding=0.025,
                              title='{} ({})'.format(varname,title)).opts(invert_xaxis=invert_x, invert_yaxis=True)
        plist += p

    return (plist + m).opts(shared_axes=False).cols(2)

In [ ]:
df_sec1 = df_glider[(df_glider.Cast > 17616000) & (df_glider.Cast < 17619000)]
df_sec1 = get_distance(df_sec1)
plot_sections(df_sec1)

In [ ]:
df_sec2 = df_glider[(df_glider.Cast > 17619000) & (df_glider.Cast < 17620700)]
df_sec2 = get_distance(df_sec2)
psec2 = plot_sections(df_sec2, invert_x=False)
psec2

In [ ]:
hvplot.save(psec2, 'figures/section_02.png') # save figure
# hv.save(psec2, 'figures/section_02.png', fmt='png')

In [ ]:
df_sec3 = df_glider[(df_glider.Cast > 17620700) & (df_glider.Cast < 17622000)]
df_sec3 = get_distance(df_sec3)
plot_sections(df_sec3)

## Glider lines south of Cape Cod (Winter)

In [ ]:
df = df_glider[df_glider.season=='winter']
df = df[(df.Latitude<42) & (df.Latitude>39.5)]

df.hvplot.points('Longitude', 'Latitude', geo=True, xlabel='longitude', ylabel='latitude', title='Glider data',
                 tiles=tiles, frame_height=450, rasterize=False, c='TA', clabel='Alkalinity', cmap='kbc', colorbar=True,
                 hover_cols=['Cast','Date'], padding=(1, 0.5))

In [ ]:
df_sec4 = df_glider[(df_glider.Cast >= 17606420) & (df_glider.Cast <= 17607136)]
df_sec4 = get_distance(df_sec4)
plot_sections(df_sec4)

In [ ]:
df_sec5 = df_glider[(df_glider.Cast >= 19106253) & (df_glider.Cast <= 19106618)]
df_sec5 = get_distance(df_sec5)
plot_sections(df_sec5)

## Glider lines south of Cape Cod (Summer)

In [ ]:
df_sec6 = df_glider[(df_glider.Cast >= 20344520) & (df_glider.Cast <= 20345580)]
df_sec6 = get_distance(df_sec6)
plot_sections(df_sec6, invert_x=False)

In [ ]:
df_sec7 = df_glider[(df_glider.Cast >= 18941091) & (df_glider.Cast <= 18941796)]
df_sec7 = df_sec7[df_sec7.Latitude>38]
df_sec7 = get_distance(df_sec7)
plot_sections(df_sec7)

In [ ]:
df_sec8 = df_glider[(df_glider.Cast >= 19650104) & (df_glider.Cast <= 19650909)]
df_sec8 = get_distance(df_sec8)
plot_sections(df_sec8)

In [ ]:
# df_sec9 = df_wod[(df_wod.Cast >= 16611177) & (df_wod.Cast <= 16612991)]
# df_sec9 = df_sec9[(df_sec9.Latitude<42) & (df_sec9.Latitude>40)]
# df_sec9 = get_distance(df_sec9)
# plot_sections(df_sec9, invert_x=False)

In [ ]:
df_summer = pd.concat([df_sec6, df_sec7, df_sec8])
df_summer.hvplot.points('Longitude', 'Latitude', geo=True, xlabel='longitude', ylabel='latitude', title='Summer glider data',
                        tiles=tiles, frame_height=450, rasterize=False, c='TA', clabel='Alkalinity', cmap='kbc', colorbar=True,
                        hover_cols=['Cast','Date'])

In [ ]:
# df = df_glider[(df_glider.season=='summer') | (df_glider.season=='fall')]
# # df = df_glider[df_glider.Date.dt.month.isin([6,7])]
# df = df[(df.Latitude<42) & (df.Latitude>39.5) & (df.Longitude>-72)]
# # df.hvplot.points('Longitude', 'Latitude', geo=True, xlabel='longitude', ylabel='latitude', title='Glider data',
# #                  tiles=tiles, frame_height=450, rasterize=True, c='TA', clabel='Alkalinity', cmap='kbc', colorbar=True,
# #                  hover_cols=['Cast','Date'])

# df.Date.dt.year.hvplot.hist(frame_width=400, bins=50) + df.Cast.hvplot.hist(frame_width=400, bins=50)